In [25]:
from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresSaver,
)
from src.config import env
from langchain_core.runnables import RunnableConfig
from langchain.load import dumpd

In [3]:
async def get_ckeckpoint():
    engine = await PostgresEngine.afrom_instance(
        project_id=env.PROJECT_ID,
        region=env.LOCATION,
        instance=env.INSTANCE,
        database=env.DATABASE,
        user=env.DATABASE_USER,
        password=env.DATABASE_PASSWORD,
        engine_args={"pool_pre_ping": True, "pool_recycle": 300},
    )
    checkpointer = await PostgresSaver.create(engine=engine)
    return checkpointer

In [ ]:
user_id = "1154af1a-7bf6-441d-ae06-4590a66c0d3d"
checkpointer = await get_ckeckpoint()
config = {"configurable": {"thread_id": user_id}}

In [ ]:
config = RunnableConfig(configurable={"thread_id": user_id})

In [11]:
l = await checkpointer.aget(config=config)

In [ ]:
messages = l["channel_values"]["messages"]

In [ ]:
from enum import Enum
from typing import Sequence
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage


class ErrorCode(Enum):
    GRAPH_RECURSION_LIMIT = "GRAPH_RECURSION_LIMIT"
    INVALID_CONCURRENT_GRAPH_UPDATE = "INVALID_CONCURRENT_GRAPH_UPDATE"
    INVALID_GRAPH_NODE_RETURN_VALUE = "INVALID_GRAPH_NODE_RETURN_VALUE"
    MULTIPLE_SUBGRAPHS = "MULTIPLE_SUBGRAPHS"
    INVALID_CHAT_HISTORY = "INVALID_CHAT_HISTORY"


def create_error_message(*, message: str, error_code: ErrorCode) -> str:
    return (
        f"{message}\n"
        "For troubleshooting, visit: https://python.langchain.com/docs/"
        f"troubleshooting/errors/{error_code.value}"
    )


def _validate_chat_history(
    messages: Sequence[BaseMessage],
) -> None:
    """Validate that all tool calls in AIMessages have a corresponding ToolMessage."""
    all_tool_calls = [
        tool_call
        for message in messages
        if isinstance(message, AIMessage)
        for tool_call in message.tool_calls
    ]
    tool_call_ids_with_results = {
        message.tool_call_id for message in messages if isinstance(message, ToolMessage)
    }
    tool_calls_without_results = [
        tool_call
        for tool_call in all_tool_calls
        if tool_call["id"] not in tool_call_ids_with_results
    ]

    print(tool_calls_without_results)
    if not tool_calls_without_results:
        return

    error_message = create_error_message(
        message="Found AIMessages with tool_calls that do not have a corresponding ToolMessage. "
        f"Here are the first few of those tool calls: {tool_calls_without_results[:3]}.\n\n"
        "Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage "
        "(result of a tool invocation to return to the LLM) - this is required by most LLM providers.",
        error_code=ErrorCode.INVALID_CHAT_HISTORY,
    )
    raise ValueError(error_message)

In [31]:
_validate_chat_history(messages)

ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'google_search', 'args': {'query': 'autorização van escolar prefeitura rio de janeiro requisitos'}, 'id': '3e66bc30-16b6-47dd-a598-048e3510fea3', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY

In [32]:
messages[1]

AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "autoriza\\u00e7\\u00e3o van escolar prefeitura rio de janeiro requisitos"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 4482, 'candidates_token_count': 13, 'total_token_count': 4597, 'cached_content_token_count': 4366, 'prompt_tokens_details': [{'modality': 1, 'token_count': 4482}], 'cache_tokens_details': [{'modality': 1, 'token_count': 4366}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'thoughts_token_count': 102}, 'finish_reason': 'STOP', 'avg_logprobs': -1.3487435854398286, 'model_name': 'gemini-2.5-flash'}, id='run--11744d20-998e-4fab-82e7-cd352e3120e6-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'autorização van escolar prefeitura rio de janeiro requisitos'}, 'id': '3e66bc30-16b6-47dd-a598-048e3510fea3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 4482, 'output_t

In [ ]:
# list = checkpointer.alist(config={"configurable": {"thread_id": user_id}})
# print(list)